In [313]:
from bs4 import BeautifulSoup
import pandas as pd


In [314]:
with open('data_capitalization.txt', 'r', encoding='utf-8') as file:
    text = file.read()


In [315]:
soup = BeautifulSoup(text, 'html.parser')
table = soup.find('tbody').find_all('td')


In [316]:
rank = []
name = []
code = []
market_cap = []
price = []
today = []
country = []


In [317]:
new = []
for row in table:
    a = row.text
    b = row.find('div', {'class': 'company-name'})
    c = row.find('div', {'class': 'company-code'})

    if b:
        name.append(b.text)

    if c:
        code.append(c.text)

    splitted = a.split('\n')
    new.append(splitted)

new = [element for element in new if '' not in element]


In [318]:
for i, data in enumerate(new):
    if i % 6 == 0:
        rank.append(data[0])

    elif i % 6 == 2:
        market_cap.append(data[0])

    elif i % 6 == 3:
        price.append(data[0])

    elif i % 6 == 4:
        today.append(data[0])

    elif i % 6 == 5:
        country.append(data[0])


In [319]:
df = pd.DataFrame([rank, name, code, market_cap, price, today, country]).transpose()


In [320]:
df.columns = ['Rank', 'Name', 'Code', 'Market_Cap', 'Price', 'Today', 'Country']


In [321]:
df.Rank = df.Rank.astype(int)
df.Price = df.Price.str.replace('$', '').str.replace(',', '').astype(float)
df.Today = df.Today.str.replace('%', '').astype(float)
df.Market_Cap = df.Market_Cap.str.replace('$', '')


In [322]:
df['multiplier'] = df.Market_Cap.str.split(' ')
df2 = pd.DataFrame(df['multiplier'].to_list(), columns=['Market_cap', 'multiplier'])
df2.multiplier = df2['multiplier'].map({'T': 1000000000000, 'B': 1000000000})
df.Market_Cap = df2.Market_cap.astype('float') * df2.multiplier
df = df.drop('multiplier', axis=1)
df

,Rank,Name,Code,Market_Cap,Price,Today,Country
0,1,NVIDIA,NVDA,4.022000e+12,164.92,0.50,🇺🇸 USA
1,2,Microsoft,MSFT,3.740000e+12,503.32,0.37,🇺🇸 USA
2,3,Apple,AAPL,3.153000e+12,211.16,0.59,🇺🇸 USA
3,4,Amazon,AMZN,2.388000e+12,225.02,1.24,🇺🇸 USA
4,5,Alphabet (Google),GOOG,2.192000e+12,181.31,1.46,🇺🇸 USA
...,...,...,...,...,...,...,...
95,96,Charles Schwab,SCHW,1.671000e+11,91.97,1.15,🇺🇸 USA
96,97,Thermo Fisher Scientific,TMO,1.641500e+11,434.86,0.96,🇺🇸 USA
97,98,Citigroup,C,1.619800e+11,86.73,0.40,🇺🇸 USA
98,99,S&P Global,SPGI,1.616400e+11,527.07,0.52,🇺🇸 USA


In [323]:
df.to_excel('data.xlsx', index=False)
